In [1]:
import pandas as pd
import numpy as np

/usr/local/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
pd.set_option('max_rows', 30)
pd.set_option('max_colwidth', 70)

In [3]:
import ddf_utils.ddf_reader as ddf

In [4]:
mapping = pd.read_csv('./gw_wdi_new.csv')

ddf.SEARCH_PATH = '/Users/semio/src/work/Gapminder'

# IHME Life expectancy

problems:

1. there are 3 life expectancy data in GW. All of 3 have same data except that the `life_expectancy_at_birth_data_from_ihme` begins from 1990 while others two begin from 1970.
2. Country Code in lhme data set is wrong. will fix it below and update README

In [11]:
cb = pd.read_csv('/Users/semio/src/work/Gapminder/ddf--ihme-lex/etl/source/IHME_GBD_2013_LIFE_EXPECTANCY_1970_2013_Y2014M12D17/IHME_GBD_2013_LIFE_EXPECTANCY_1970_2013_CB_Y2014M12D17.CSV', skiprows=1)

In [8]:
data = pd.read_csv('/Users/semio/src/work/Gapminder/ddf--ihme-lex/etl/source/IHME_GBD_2013_LIFE_EXPECTANCY_1970_2013_Y2014M12D17/IHME_GBD_2013_LIFE_EXPECTANCY_1970_2013_Y2014M12D17.CSV')

In [9]:
data

,location_id,location_code,location_name,year,age_group_id,age_group_name,sex_id,sex_name,metric,unit,mean,upper,lower
0,1,G,Global,1970,28,<1 year,1,Males,Life expectancy,Years,56.7,57.6,55.7
1,1,G,Global,1970,28,<1 year,2,Females,Life expectancy,Years,60.1,61.1,59.0
2,1,G,Global,1970,28,<1 year,3,Both sexes,Life expectancy,Years,58.4,59.0,57.7
3,1,G,Global,1970,5,1 to 4,1,Males,Life expectancy,Years,61.8,62.8,60.7
4,1,G,Global,1970,5,1 to 4,2,Females,Life expectancy,Years,65.0,65.9,64.0


In [13]:
cb = cb.drop(0)

In [14]:
cb.head()

,Variable:,location_id,location_code,location_name,year,age_group_id,age_group_name,sex_id,sex_name,metric,unit,mean,upper,lower
1,Value coding:,1,G,Global,1970,28,<1 year,1,Males,Life expectancy,Years,NaN,NaN,NaN
2,NaN,2,D0,Developing,1971,5,1 to 4,2,Females,NaN,NaN,NaN,NaN,NaN
3,NaN,3,D1,Developed,1972,6,5 to 9,3,Both sexes,NaN,NaN,NaN,NaN,NaN
4,NaN,4,S6,"Southeast Asia, East Asia, and Oceania",1973,7,10 to 14,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,5,R3,East Asia,1974,8,15 to 19,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
d1 = data[['location_id', 'location_name', 'location_code']].drop_duplicates().reset_index(drop=True)

In [55]:
d2 = cb[['location_id', 'location_name', 'location_code']].drop_duplicates().reset_index(drop=True)

In [56]:
assert np.all(d2['location_code'] == d1['location_code'])

In [60]:
np.all(d1['location_name'] == d2['location_name'])

True

In [58]:
d2['location_name'] = d1['location_name']

In [73]:
cb = cb.reset_index(drop=True)

In [74]:
cb['location_name'] = d2['location_name']

In [76]:
cb['location_name'].to_csv('tmp.csv')

# Here comes the mappings

In [78]:
# country code mappings

cgw = ddf.ddf_entities('ddf--gapminder--gapminder_world')['country']

In [81]:
cgw = cgw.set_index('name')

In [106]:
clex = ddf.ddf_entities('ddf--ihme-lex')['location_id']
clex['location_id'] = clex['location_id'].map(str)

In [107]:
clex = clex.set_index('location_name')

In [87]:
cgw[['country']]

,country
name,
Abkhazia,abkh
Afghanistan,afg
Akrotiri and Dhekelia,akr_a_dhe
Albania,alb
Algeria,dza
American Samoa,asm
Andorra,and
Angola,ago
Anguilla,aia


In [108]:
clex2 = clex[~clex['location_code'].str.match('\w\d+')].drop('Global').copy()

In [99]:
comb = set(clex2.index).union(set(cgw.index))

In [110]:
cgw2 = cgw[['country']].ix[comb].copy()

In [111]:
cgw2 = cgw2.sort_index()

In [112]:
cgw2['ihme_id'] = clex2['location_id']
cgw2['ihme_code'] = clex2['location_code']

In [113]:
cgw2

,country,ihme_id,ihme_code
name,,,
Abkhazia,abkh,NaN,NaN
Afghanistan,afg,160,AFG
Akrotiri and Dhekelia,akr_a_dhe,NaN,NaN
Albania,alb,43,ALB
Algeria,dza,139,DZA
American Samoa,asm,NaN,NaN
Andorra,and,74,AND
Angola,ago,168,AGO
Anguilla,aia,NaN,NaN


In [115]:
tmp = cgw2.dropna(how='any')

In [120]:
np.all(tmp['ihme_code'].str.lower() == tmp['country'])  ### no convertion needed, all codes are same.

True

In [5]:
filt2 = mapping['concept'].str.contains('life_expectancy').fillna(False)

In [6]:
mapping[filt2]  # notice that 3 of the source links are the same one.

,concept,settings_link,wdi_id,found_in_wdi
43,life_expectancy_years,http://spreadsheets.google.com/pub?key=phAwcNAVuyj2tPLxKvvnNPA&gid=1,NaN,NaN
488,data_quality_life_expectancy,http://www.gapminder.org/data/documentation/gd004/,NaN,NaN
496,life_expectancy_at_birth_with_projections,http://spreadsheets.google.com/pub?key=phAwcNAVuyj2tPLxKvvnNPA&gid=1,NaN,NaN
556,life_expectancy_at_birth_data_from_ihme,http://spreadsheets.google.com/pub?key=phAwcNAVuyj2tPLxKvvnNPA&gid=1,NaN,NaN
559,life_expectancy_male,http://esa.un.org/unpd/wpp/Excel-Data/Interpolated.htm,NaN,NaN
560,life_expectancy_female,http://esa.un.org/unpd/wpp/Excel-Data/Interpolated.htm,NaN,NaN


In [121]:
# check if ssd and sdn is same in both

ddf.ddf_datapoint('ddf--gapminder--gapminder_world', 'life_expectancy_years').query(
    'time in [1970, 1980, 1990] & geo in ["zwe"]')

,geo,life_expectancy_years,time
43172,zwe,57.9,1970
43182,zwe,59.8,1980
43192,zwe,63.3,1990


In [122]:
cgw2.query('ihme_code == "ZWE"')

,country,ihme_id,ihme_code
name,,,
Zimbabwe,zwe,198,ZWE


In [123]:
# sex_id == 3 & age_group_id == 28 means both sexes and age under 1(new birth)

ddf.ddf_datapoint('ddf--ihme-lex', 'mean').query(
    'year in [1970, 1980, 1990] & location_id in [198] & sex_id == 3 & age_group_id == 28')

,location_id,year,age_group_id,sex_id,mean
620930,198,1970,28,3,57.9
621650,198,1980,28,3,59.8
622370,198,1990,28,3,63.3
